In [5]:
# Import libraries
from pyomo.environ import *
from pyomo.opt import SolverFactory


In [6]:
# Create a solver
opt = SolverFactory('gurobi')

#
# A simple model with binary variables and
# an empty constraint list.
#
model = AbstractModel()
model.n = Param(default=4)
model.x = Var(RangeSet(model.n), within=Binary)


def o_rule(model):
    return summation(model.x)


model.o = Objective(rule=o_rule)
model.c = ConstraintList()

# Create a model instance and optimize
instance = model.create_instance()
results = opt.solve(instance)
instance.display()

# Iterate to eliminate the previously found solution
for i in range(5):
    expr = 0
    for j in instance.x:
        if value(instance.x[j]) == 0:
            expr += instance.x[j]
        else:
            expr += 1 - instance.x[j]
    instance.c.add(expr >= 1)
    results = opt.solve(instance)
    print("\n===== iteration", i)
    instance.display()

Model unknown

  Variables:
    x : Size=4, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :   0.0 :     1 : False : False : Binary
          2 :     0 :   0.0 :     1 : False : False : Binary
          3 :     0 :   0.0 :     1 : False : False : Binary
          4 :     0 :   0.0 :     1 : False : False : Binary

  Objectives:
    o : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True :   0.0

  Constraints:
    c : Size=0
        Key : Lower : Body : Upper

===== iteration 0
Model unknown

  Variables:
    x : Size=4, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :   0.0 :     1 : False : False : Binary
          2 :     0 :   0.0 :     1 : False : False : Binary
          3 :     0 :   0.0 :     1 : False : False : Binary
          4 :     0 :   1.0 :     1 : False : False : Binary

  Objectives:
    o : Size=1, Index=None, Active=True
        Key  : Acti

In [12]:
# Create the model object
model = ConcreteModel()

# Declaring the decision variables
model.A = Var(within=NonNegativeReals) # number of product A
model.B = Var(within=NonNegativeReals) # number of product B

# Objective function
model.max_profit = Objective(expr = 200*model.A + 300*model.B, sense = maximize)

# Constraints
model.A_cons = Constraint(expr = 3 * model.A + 2 * model.B <= 100)
model.B_cons = Constraint(expr = 2 * model.A + 4 * model.B <= 120)
model.C_cons = Constraint(expr =  model.A + model.B <=45)

# Run model 
optimizer = SolverFactory("gurobi")
optimizer.solve(model)
model.display()

Model unknown

  Variables:
    A : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  20.0 :  None : False : False : NonNegativeReals
    B : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  20.0 :  None : False : False : NonNegativeReals

  Objectives:
    max_profit : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 10000.0

  Constraints:
    A_cons : Size=1
        Key  : Lower : Body  : Upper
        None :  None : 100.0 : 100.0
    B_cons : Size=1
        Key  : Lower : Body  : Upper
        None :  None : 120.0 : 120.0
    C_cons : Size=1
        Key  : Lower : Body : Upper
        None :  None : 40.0 :  45.0
